In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.neighbors import KDTree

In [25]:
df=pd.read_csv("submission.csv")
df=df[['event_id', 'hit_id', 'track_id']]

In [26]:
root="/home/alexanderliao/data/Kaggle/competitions/trackml-particle-identification/test/"

for i in tqdm(range(125)):
    if i==0:
        hits=pd.read_csv(root+"event"+str(i).zfill(9)+"-hits.csv")
    else:
        temp=pd.read_csv(root+"event"+str(i).zfill(9)+"-hits.csv")
        hits=pd.concat([hits,temp])

100%|██████████| 125/125 [01:02<00:00,  2.00it/s]


In [27]:
hits[['hit_id', 'x','y','z']].to_csv("feature.csv",index=False)

In [28]:
hits=hits[['hit_id','x','y','z']]

In [29]:
df.iloc[119016]
hits

,hit_id,x,y,z
0,1,-60.8267,-4.160230,-1502.5
1,2,-84.7294,-7.295280,-1502.5
2,3,-42.5930,5.048750,-1502.5
3,4,-62.4975,2.729920,-1502.5
4,5,-66.2037,-10.601200,-1502.5
5,6,-63.0563,-3.073910,-1502.5
6,7,-100.0770,-9.506260,-1502.5
7,8,-56.6046,-1.169750,-1502.5
8,9,-60.5920,2.177720,-1502.5
9,10,-56.1216,-9.457400,-1502.5


In [30]:
event1=df.loc[df['event_id'] == 0]
event1.shape[0]

119016

In [31]:
pre=pd.concat([df.loc[df['event_id'] == 0],hits.iloc[0:event1.shape[0]]],axis=1)

In [33]:
index = 0
for i in tqdm(range(125)):
    event = df.loc[df['event_id'] == i]
    temp = pd.merge(event,hits.iloc[index:index+event.shape[0]],on='hit_id')
    index=index+event.shape[0]
    #print(index)
    temp[['event_id','hit_id', 'track_id','x','y','z']].to_csv("./events/event"+str(i).zfill(3)+".csv",index=False)

100%|██████████| 125/125 [01:15<00:00,  1.67it/s]


In [34]:
index

13741466

In [35]:
def extend(df,limit=0.04, num_neighbours=18):

    #df = submission.merge(hits,  on=['hit_id'], how='left')
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-90,90,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.5)/180*np.pi) & (df.arctan2<(angle+1.5)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<3: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1[['x', 'y', 'z']].values.T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        c = np.cos(a)
        s = np.sin(a)
        #tree = KDTree(np.column_stack([a,r]), metric='euclidean')
        tree = KDTree(np.column_stack([c, s, r]), metric='euclidean')


        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]
            c0 = c[idx0]
            c1 = c[idx1]
            s0 = s[idx0]
            s1 = s[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0)

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1)



            ## extend start point
            ns = tree.query([[c0, s0, r0]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r0 - r[ns], a0 - a[ns])
            diff = 1 - np.cos(direction - direction0)
            ns = ns[(r0 - r[ns] > 0.01) & (diff < (1 - np.cos(limit)))]
            for n in ns: df.loc[df.hit_id == hit_ids[n], 'track_id'] = p

            ## extend end point
            ns = tree.query([[c1, s1, r1]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns] - r1, a[ns] - a1)
            diff = 1 - np.cos(direction - direction1)
            ns = ns[(r[ns] - r1 > 0.01) & (diff < (1 - np.cos(limit)))]
            for n in ns:  df.loc[df.hit_id == hit_ids[n], 'track_id'] = p

    #print ('\r')
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [ ]:
for i in tqdm(range(125)):
    result=pd.read_csv("./events/event"+str(i).zfill(3)+".csv")
    #print("before:{}".format(result.shape[0]))
    result=extend(result,limit=0.04, num_neighbours=18)
    #print("after:{}".format(result.shape[0]))
    result[['event_id', 'hit_id', 'track_id']].to_csv("./enhanced/enhanced"+str(i).zfill(3)+".csv",index=False)

  0%|          | 0/125 [00:00<?, ?it/s]

 89.0000000

  1%|          | 1/125 [00:39<1:20:43, 39.06s/it]

 89.0000000

  2%|▏         | 2/125 [01:20<1:22:11, 40.10s/it]

 89.0000000

  2%|▏         | 3/125 [01:55<1:18:33, 38.64s/it]

 89.0000000

  3%|▎         | 4/125 [02:31<1:16:29, 37.93s/it]

 -84.000000

In [ ]:
result=pd.read_csv("./events/event"+str(2).zfill(3)+".csv")

In [ ]:
result

In [ ]:
#del temp
#del final
for i in tqdm(range(125)):
    temp=pd.read_csv("./enhanced/enhanced"+str(i).zfill(3)+".csv")
    temp=temp[['event_id', 'hit_id', 'track_id']]
    print(temp.shape[0])
    print(i)
    if i==0:
        final=temp
    else:
        final=pd.concat([final,temp])
        #print(final.shape[0])

In [ ]:
final

In [ ]:
final[['event_id', 'hit_id', 'track_id']]=final[['event_id', 'hit_id', 'track_id']].fillna(0).astype('int') 
final.to_csv('./enhanced.csv',index=False)


In [ ]:
a=pd.read_csv("enhanced.csv")

In [ ]:
a